In [59]:
import os
import glob
import json
import numpy as np
import matplotlib.pyplot as plt

import scipy.sparse as sps
import scipy.linalg as sl
from sksparse.cholmod import cholesky

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
import enterprise.constants as const
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import utils
from enterprise.signals import gp_signals

In [60]:
# Set the data and noise locations
datadir = '/home/marvin/NANOGrav/perm-data/9yr/'
noisedir = '/home/marvin/NANOGrav/perm-data/noise/'

In [62]:
parfiles = sorted(glob.glob(datadir+'par/'+'*.par'))
timfiles = sorted(glob.glob(datadir+'tim/'+'*.tim'))
print(parfiles)

['/home/marvin/NANOGrav/perm-data/9yr/par/B1855+09_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/B1937+21_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/B1953+29_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J0023+0923_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J0030+0451_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J0340+4130_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J0613-0200_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J0645+5158_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J0931-1902_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J1012+5307_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J1024-0719_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J1455-3330_NANOGrav_9yv1.gls.par', '/home/marvin/NANOGrav/perm-data/9yr/par/J1600-3053_NANOGrav_9yv1.gls.par', '/home/marvin/NAN

In [128]:
psr = Pulsar(parfiles[3], timfiles[3], ephem='DE436')

In [129]:
# For now, let's just add EFAC
efac = parameter.Constant()
equad = parameter.Constant()

selection = selections.Selection(selections.by_backend)

ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)

In [130]:
# Now we'll try to add red noise
log10_A = parameter.Constant()
gamma = parameter.Constant()

# define powerlaw PSD and red noise signal
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(pl, components=30, name='rn')

In [131]:
# Set the full signal, which is just the EFAC here
s = ef + eq + rn

In [132]:
# Need to hardcode the 'B' names for some reason
# initialize PTA
model = [s(psr)]
pta = signal_base.PTA(model)

In [133]:
noisefile = noisedir + 'noisedict.json'
setpars = {}
with open(noisefile, 'r') as fin:
    setpars.update(json.load(fin))

# fix the white noise parameters to the values in the noisefiles
pta.set_default_params(setpars)

INFO: enterprise.signals.signal_base: Setting J0023+0923_430_ASP_efac to 1.046
INFO: enterprise.signals.signal_base: Setting J0023+0923_430_PUPPI_efac to 1.04089
INFO: enterprise.signals.signal_base: Setting J0023+0923_L-wide_ASP_efac to 1.15689
INFO: enterprise.signals.signal_base: Setting J0023+0923_L-wide_PUPPI_efac to 1.16122
INFO: enterprise.signals.signal_base: Setting J0023+0923_430_ASP_log10_equad to -9.96299
INFO: enterprise.signals.signal_base: Setting J0023+0923_430_PUPPI_log10_equad to -6.91689
INFO: enterprise.signals.signal_base: Setting J0023+0923_L-wide_ASP_log10_equad to -5.90181
INFO: enterprise.signals.signal_base: Setting J0023+0923_L-wide_PUPPI_log10_equad to -6.71605
INFO: enterprise.signals.signal_base: Setting J0023+0923_rn_log10_A to -13.1929
INFO: enterprise.signals.signal_base: Setting J0023+0923_rn_gamma to 0.0896611


In [134]:
def get_params(noisefile, psr):
    pars = {}
    with open(noisefile, 'r') as f:
        pars.update(json.load(f))
    
    params = {k:v for (k,v) in pars.items() if k.startswith(psr.name) and not k.endswith('ecorr')}
    return params

In [135]:
def construct_N(sc, psr):
    # Function to construct white noise matrix `N`
    # For now it only accomadates single pulsar PTAs
    
    # W matrix is a diagonal matrix of TOA uncertainties squared
    W = np.diag(psr.toaerrs) ** 2

    # Separate out different types of white noise
    ef_noise = sc._signals[0]
    eq_noise = sc._signals[1]

    # Let's divide up into our different observing systems (our k's)
    # First we need to generate a 'mask' dictionary, aka matching receivers
    # to the TOAs they measured
    sel = selection(psr)
    masks = sel.masks

    ef_vals = [l.value**2 for l in list(ef_noise._params.values())]
    ef_dict = dict(zip(ef_noise._keys, ef_vals))

    eq_vals = [10**(2*l.value) for l in list(eq_noise._params.values())]
    eq_dict = dict(zip(eq_noise._keys, eq_vals))

    # Now we construct the N_k dictionary, one N matrix per backend+receiver combo
    # Then sum them together to get the final N
    N_k = {}
    for key, mask in masks.items():
        N_k[key] = (ef_dict[key] * W + eq_dict[key]*np.identity(len(psr.toas))) * mask

    N = sum(N_k.values())
    
    return N

In [136]:
def construct_T(psr):
    Ti = psr.toas.max() - psr.toas.min()
    fmin = 1/Ti
    fmax = 30/Ti
    f = np.linspace(fmin, fmax, 30)
    ranphase = np.zeros(30)
    
    Ffreqs = np.repeat(f, 2)
    N = len(psr.toas)
    F = np.zeros((N, 2*30))
    
    F[:, ::2] = np.sin(2 * np.pi * psr.toas[:, None] * f[None, :] + ranphase[None, :])
    F[:, 1::2] = np.cos(2 * np.pi * psr.toas[:, None] * f[None, :] + ranphase[None, :])
    
    basis = F, Ffreqs
    return basis

In [137]:
def powerlaw(f, log10_A=-16, gamma=5, components=2):
    df = np.diff(np.concatenate((np.array([0]), f[::components])))
    return (
        (10 ** log10_A) ** 2 / 12.0 / np.pi ** 2 * const.fyr ** (gamma - 3) * f ** (-gamma) * np.repeat(df, components)
    )

def construct_phi(sc, psr, params):
    
    T, Ffreqs = construct_T(psr)
    phi = KernelMatrix(sc._Fmat.shape[1])
    
    rnsig = sc._signals[2]
    basis, labels = {}, {}
    for key, mask in zip(rnsig._keys, rnsig._masks):
        basis[key], labels[key] = rnsig._bases[key](params=params, mask=mask)

    nc = sum(F.shape[1] for F in basis.values())
    rnbasis = np.zeros((len(rnsig._masks[0]), nc))
    rnslices = {}
    nctot = 0
    
    for key, mask in zip(rnsig._keys, rnsig._masks):
        Fmat = basis[key]
        nn = Fmat.shape[1]
        rnbasis[mask, nctot : nn + nctot] = Fmat
        rnslices.update({key: slice(nctot, nn+nctot)})
        nctot += nn
    
    log10A = params[psr.name+'_rn_log10_A']
    gam = params[psr.name+'_rn_gamma']
    
    for key, slc in rnslices.items():
        phislc = powerlaw(Ffreqs, log10_A=log10A, gamma=gam)
        phi = phi.set(phislc, slc)
    
    return phi

In [138]:
def get_TNr(T, Nvec, r):
    mult = np.array(r/Nvec)
    TNr = np.dot(T.T, mult)
    return TNr

def get_TNT(T, Nvec):
    TNT = np.dot(T.T, np.array(T/Nvec[:, None]))
    return TNT

def get_rT_Ninv_r(r, N):
    rT = np.transpose(r)
    Ninv = np.linalg.inv(N)
    
    rT_Ninv_r = np.dot(rT, np.dot(Ninv, r))
    return rT_Ninv_r

def get_phiinv(phi):
    phiinv = 1.0 / phi
    logdet_phi = np.sum(np.log(phi))
    
    return phiinv, logdet_phi

In [139]:
def custom_likelihood(pta, psr, params):
    # Currently this function only works for a single pulsar
    # it will be updated in the future to adapt to multiple-pulsar PTAs
    ln_likelihood = 0
    
    sc = pta.pulsarmodels[0]
    
    r = psr.residuals
    
    N = construct_N(sc, psr)
    Nvec = np.diag(N)
    logdet_N = np.sum(np.log(Nvec))
    
    T, Ffreqs = construct_T(psr)
    
    phi = construct_phi(sc, psr, params)
    phiinv, logdet_phi = get_phiinv(phi)

    TNr = get_TNr(T, Nvec, r)
    TNT = get_TNT(T, Nvec)
    rT_Ninv_r = get_rT_Ninv_r(r, N)
    
    Sigma = TNT + (np.diag(phiinv) if phiinv.ndim == 1 else phiinv)
    cf = sl.cho_factor(Sigma)
    expval = sl.cho_solve(cf, TNr)
    logdet_sigma = np.sum(2 * np.log(np.diag(cf[0])))
    
    ln_likelihood += -0.5 * (rT_Ninv_r + logdet_N)
    ln_likelihood += 0.5 * (np.dot(TNr, expval) - logdet_sigma - logdet_phi)
    return ln_likelihood

In [140]:
params = get_params(noisefile, psr)
loglike = custom_likelihood(pta, psr, params)
print(pta.get_lnlikelihood(params))
print(loglike)

52577.37883497028
52577.37883497028
